<a href="https://colab.research.google.com/github/Ferrariagustinpablo/Python-Machine-Learning-notebooks/blob/main/Hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_csv('https://assets.datacamp.com/production/repositories/3983/datasets/bb158f1c76682286f938e02d71de21a3e5389cbf/credit-card-full.csv')

print(df.shape)
df.head()

(30000, 25)


,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0


# What is Hyperparameter tuning

Somthing you set before the the modelling process.

The algorithm does not learn the value of hyperparameters.

Some hyperparameters are more important than others. There are some generally accepted important hyperparameters to tune.


## Random Forest

The most import hyperparameters on a Random Forest:

n_estimators (should be set to a high value)

max_features (how many features to consider when splitting, vital to ensure tree diversity)

max_depth & min_sample_leaf (important to control overfitting)

criterion: not a generally a primary hyperparameter to consider.

Understanding what hyperparameters are available and the impact of different hyperparameters is a core skill for any data scientist. As models become more complex, there are many different settings you can set, but only some will have a large impact on your model.

In [ ]:

<script.py> output:
    RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                max_depth=None, max_features='auto', max_leaf_nodes=None,
                min_impurity_decrease=0.0, min_impurity_split=None,
                min_samples_leaf=1, min_samples_split=2,
                min_weight_fraction_leaf=0.0, n_estimators=5, n_jobs=None,
                oob_score=False, random_state=42, verbose=0, warm_start=False)

##  Hyperparameters of KNN

The k-nearest-neighbors algorithm is not as popular as it used to be but can still be an excellent choice for data that has groups of data that behave similarly. Could this be the case for our credit card users?

## LogisticRegression

For example in logistic regression parameter solver gets in conflictor with penalty.

# GridSearchCV

Scikit-learn GridSearchCV

Steps in GridSearchCV:

1- An algorithm to tune the hyperparameters (estimator)
2- Defining which hyperparameters we will tune.
3- Defining a range of values for each hyperparameter
4- Setting a cross-validation scheme
5- Define a score function so we can decide which square on our grid is the 'best'



Important arguments in GridSearchCV

1- estimator: KNN, Logregression, GBM, RandomForest, and others

2- param_grid: dictionary of hyperparameter names and list of values. (The keys must be valid hyperparameters)

3- cv: provide an integer for cross-validation. Typically 5 or 10

4- scoring: Which score to use to choose best grid squares. Sci-kit learn metrics module. 

5- refit: True means that it fits the best hyperparameters to the training data. This means that the GridSearchCV object transforms into an estimator where it can be used for prediction with .predict(). Really handy.

6- n_jobs: Using different CPUs

7- return_train_score: Logs statistics about the training runs. **Useful for analyzing bias-variance trade-off but adds computational expense.** Super informative but computational expensive and would not assist in finding the model, onnly for information. 

In [ ]:
# All scoring metrics from scikit-learn
from sklearn import metrics
print(sorted(metrics.SCORERS.keys()))

['accuracy', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'average_precision', 'balanced_accuracy', 'completeness_score', 'explained_variance', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'fowlkes_mallows_score', 'homogeneity_score', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted', 'max_error', 'mutual_info_score', 'neg_brier_score', 'neg_log_loss', 'neg_mean_absolute_error', 'neg_mean_gamma_deviance', 'neg_mean_poisson_deviance', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_median_absolute_error', 'neg_root_mean_squared_error', 'normalized_mutual_info_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'r2', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'roc_auc', 'roc_auc_ovo', 'roc_auc_ovo_weighted', 'roc_auc_ovr', 'roc_auc_ovr_weighted', 'v_measure_score']


In [ ]:
# Create a Random Forest Classifier with specified criterion
rf_class = RandomForestClassifier(criterion='entropy')

# Create the parameter grid
param_grid = {'max_depth': [2, 4, 8, 15], 'max_features': ['auto', 'sqrt']}

# Create a GridSearchCV object
grid_rf_class=GridSearchCV(
    estimator=rf_class,
    param_grid=param_grid,
    scoring='roc_auc',
    n_jobs=1,
    cv=5,
    refit=True, return_train_score=True)
print(grid_rf_class)

## GridSearchCV outputs

### Results log cv_results_

Dictionary that we can read into a DataFrame

In [ ]:
cv_results = pd.DataFrame(grid_rf_class.cv_results_)

### best results: best_index_, best_params_ and best_score_

best_index_: the row in our grid that was the best.

best_params_: the dictionary of parameters that gave the best score.

best_score_: the actual best score.

In [ ]:
# Print out the ROC_AUC score from the best-performing square
best_score = grid_rf_class.best_score_
print(best_score)

# Create a variable from the row related to the best-performing square
cv_results_df = pd.DataFrame(grid_rf_class.cv_results_)
best_row = cv_results_df.loc[[grid_rf_class.best_index_]]
print(best_row)

# Get the n_estimators parameter from the best-performing square and print
best_n_estimators = grid_rf_class.best_params_["n_estimators"]
print(best_n_estimators)

Using the best_estimator_ to predict

In [ ]:
# See what type of object the best_estimator_ property is
print(type(grid_rf_class.best_estimator_))

# Create an array of predictions directly using the best_estimator_ property
predictions = grid_rf_class.best_estimator_.predict(X_test)

# Take a look to confirm it worked, this should be an array of 1's and 0's
print(predictions[0:5])

# Now create a confusion matrix 
print("Confusion Matrix \n", confusion_matrix(y_test, predictions))

# Get the ROC-AUC score 
# Be careful. Use predict 
predictions_proba = grid_rf_class.best_estimator_.predict_proba(X_test)[:,1]
print("ROC-AUC Score \n", roc_auc_score(y_test, predictions_proba))

### Extra information: scorer_, n_splits_ and refit_time_

# RandomSearchCV

When it comes to undertaking the search rather than trying every single combination we randomly sample N combinations and try these out.

Why doing Random Search? Paper of Bengio & Bergstra (2012), shows empirically and theoretically that randomly chosen trials are more efficient for hyperparameter optimization than trials on a grid.

Two main reasons:

1- Not every hyperparameter is as important

2- Probability. How many trials to have a 95% confidence of getting the region of best parameters. The probability of missing is (1-0.05)^n.

## Additionally we can create a random sample of hyperparameters

To undertake a random search, we firstly need to undertake a random sampling of our hyperparameter space.

 firstly create some lists of hyperparameters that can be zipped up to a list of lists. Then you will randomly sample hyperparameter combinations preparation for running a random search.

In [ ]:
from itertools import product

# Create a list of values for the learning_rate hyperparameter
learn_rate_list = list(np.linspace(0.01,1.5,200))

# Create a list of values for the min_samples_leaf hyperparameter
min_samples_list = list(range(10,41))

# Combination list
# product from itertools
combinations_list = [list(x) for x in product(learn_rate_list, min_samples_list)]

# Sample hyperparameter combinations for a random search.
combinations_random_chosen = random.sample(combinations_list, 250)

# Print the result
print(combinations_random_chosen)

In [ ]:
# Create lists for criterion and max_features
criterion_list = ["gini", "entropy"]
max_feature_list = ["auto", "sqrt", "log2", None]

# Create a list of values for the max_depth hyperparameter
max_depth_list = list(range(3,56))

# Combination list
combinations_list = [list(x) for x in product(criterion_list, max_feature_list, max_depth_list)]

# Sample hyperparameter combinations for a random search
combinations_random_chosen = random.sample(combinations_list, 150)

# Print the result
print(combinations_random_chosen)

## RandomSearchCV

Only one different when undertaking random seatch 

1- Decide how many samples to take. n_iter.


And in arguments:
param_distributions if you want to tell computer how to sample.

In [ ]:
# Create the parameter grid
param_grid = {'learning_rate': np.linspace(0.1, 2, 150), 'min_samples_leaf': list(range(20, 65))} 

# Create a random search object
random_GBM_class = RandomizedSearchCV(
    estimator = GradientBoostingClassifier(),
    param_distributions = param_grid,
    n_iter = 10,
    scoring='accuracy', n_jobs=4, cv = 5, refit=True, return_train_score = True)

# Fit to the training data
random_GBM_class.fit(X_train, y_train)

# Print the values used for both hyperparameters
print(random_GBM_class.cv_results_['param_learning_rate'])
print(random_GBM_class.cv_results_['param_min_samples_leaf'])

In [ ]:
# Create the parameter grid
param_grid = {'max_depth': list(range(5,26)), 'max_features': ['auto' , 'sqrt']} 

# Create a random search object
random_rf_class = RandomizedSearchCV(
    estimator = RandomForestClassifier(n_estimators=80),
    param_distributions = param_grid, n_iter = 5,
    scoring='roc_auc', n_jobs=4, cv = 3, refit=True, return_train_score = True)

# Fit to the training data
random_rf_class.fit(X_train, y_train)

# Print the values used for both hyperparameters
print(random_rf_class.cv_results_['param_max_depth'])
print(random_rf_class.cv_results_['param_max_features'])

If you want the absolute best --> GridSearchCV

If lot of data, or many hyperparameters to tune and less resources and time --> RandomSearchCV

# Informed Search

Hyperparameter tuning techniques know as informed search

Grid and randomSearch are uninformed search. Each iteration does not learn from the previous iteration.

## **Coarse to fine tuning** Is a technique of informed search:
1- Undertake a RandomSearch.
2- Then fin promising areas
3- Grid search in the smaller area.
4- Continue until optimal score obtained.

**Uses the advantages of both methods**

Uses batches in iteration

## Bayesian Statistics

Important statistical concept.

Statistical method of using new evidence to iteratively update our beliefs about some outcome.

Bayesian hyperparameter tuning is very new but quite popular for larger and more complex hyperparameter tuning tasks as they work well to find optimal hyperparameter combinations in these situations.


Many options to set the grid of parameters:

Simple numbers, choose from a list or distribution of values.

### hyperopt

we will use hp.quniform

In [ ]:
from hyperopt import fmin, hp

# Set up space dictionary with specified hyperparameters
space = {'max_depth': hp.quniform('max_depth', 2, 10, 2),'learning_rate': hp.uniform('learning_rate', 0.001, 0.9)}

# Set up objective function
def objective(params):
    params = {'max_depth': int(params['max_depth']),'learning_rate': params['learning_rate']}
    gbm_clf = GradientBoostingClassifier(n_estimators=100, **params) 
    best_score = cross_val_score(gbm_clf, X_train, y_train, scoring='accuracy', cv=2, n_jobs=4).mean()
    loss = 1 - best_score
    return loss

# Run the algorithm
best = fmin(fn=objective,space=space, max_evals=20, rstate=np.random.RandomState(42), algo=tpe.suggest)
print(best)

This will be a very powerful tool for your machine learning modeling in future.

## Genetic Algorithms like TPOT

https://www.datacamp.com/community/tutorials/tpot-machine-learning-python

Complete tutorial on datacamp

Another informed search methodology.

1- We can create some models
2- We pick the best by our scoring function. 
3- We can create new models that are similar to the best ones. 
4- We add some randomness so we don't reach a local optimum.
5- Repeat until we are happy!

Number of advantages.
1- Allows to learn from the previous iteration like bayesian. 
2- It has additional advantage of randomness.

** It doesn't require hyperparameter choice!**

https://towardsdatascience.com/tpot-automated-machine-learning-in-python-e56800e69c11

Consider TPOT your Data Science Assistant. TPOT is a Python Automated Machine Learning tool that optimizes machine learning pipelines using genetic programming.

In a way, TPOT is your data science assistant. It automates the “boring” stuff and leaves you with the tasks of data gathering and preparation. I know this segment isn’t fun to some of you — but it’s 80% of the job. With libraries like TPOT, that percentage might only get higher.

http://geneticprogramming.com

https://epistasislab.github.io/tpot/

It returns the python code of the pipeline for you!!!!!!!

The key arguments to a TPOT classifier are:

1- Generation: iteration to run training for. Number of cycles.
2- Population_size: how many model we keeps
3- offspring_size: number of models to produce in each iteration
4- mutation_rate: The proportion of pipelines to apply randomness to.
5- cv: 

In [ ]:
from tpot import TPOTClassifier

# Assign the values outlined to the inputs
number_generations = 3
population_size = 4
offspring_size = 3
scoring_function = 'accuracy'

# Create the tpot classifier
tpot_clf = TPOTClassifier(generations=number_generations, population_size=population_size, 
                          offspring_size=offspring_size, scoring=scoring_function,
                          verbosity=2, random_state=2, cv=2)

# Fit the classifier to the training data
tpot_clf.fit(X_train, y_train)

# Score on the test set
print(tpot_clf.score(X_test, y_test))


tpot_clf.fitted_pipeline_

TPOT is quite unstable when not run for a reasonable amount of time.

quite unstable when only running with low generations, population size and offspring like in the code.

Increasing the generations, population size and offspring and running this for a long time will assist to produce better models and more stable results.

In [ ]:
pip install tpot

     |████████████████████████████████| 92kB 2.3MB/s 
     |████████████████████████████████| 163kB 8.6MB/s 
  Created wheel for stopit: filename=stopit-1.1.2-cp36-none-any.whl size=11956 sha256=6cb986dddcbdd899f74a33e0642a0e809ca11444146d9641733d4aa664ce0b01
  Stored in directory: /root/.cache/pip/wheels/3c/85/2b/2580190404636bfc63e8de3dff629c03bb795021e1983a6cc7
Successfully built stopit


In [ ]:
from tpot import TPOTClassifier

TPOTClassifier?